<a href="https://colab.research.google.com/github/mohameddhameem/TensorflowCertification/blob/main/Tensorflow_2_x_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 2.x CNN

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT = 0.3
IMG_ROWS, IMG_COLS = 28, 28
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1) # We use only 1 channel
NB_CLASSES = 10

In [2]:
#Lets build the model
def build(input_shape, classes):
  model = models.Sequential()
  # CONV => RELU => POOL
  model.add(layers.Convolution2D(20, (2,2), activation='relu', input_shape = input_shape))
  model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # CONV => RELU => POOL
  model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  # Flatten => RELU layers - before going for final classifier flatten it
  model.add(layers.Flatten())
  model.add(layers.Dense(500, activation='relu'))
  # a softmax classifier - final layer
  model.add(layers.Dense(classes, activation="softmax"))
  return model

In [3]:
# Build the data set
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
#reshape
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))
#normalize
X_train, X_test = X_train / 255.0, X_test / 255.0

#convert / cast to float 32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, NB_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NB_CLASSES)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
model = build(INPUT_SHAPE, NB_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER,
              metrics=["accuracy"])
model.summary()

#custom callback
callbacks = [
             tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
history = model.fit(X_train, y_train,
                    batch_size = BATCH_SIZE,
                    epochs = EPOCHS,
                    verbose = VERBOSE,
                    validation_split=VALIDATION_SPLIT,
                    callbacks = callbacks)
score = model.evaluate(X_test,y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 20)        100       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 50)          25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dens

# CIFAR Dataset

In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

EPOCHS = 50
NUM_CLASSES = 10

def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
 
    # normalize 
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
 
    y_train =  tf.keras.utils.to_categorical(y_train,NUM_CLASSES)
    y_test =  tf.keras.utils.to_categorical(y_test,NUM_CLASSES)
    return x_train, y_train, x_test, y_test

def build_model():
    model = models.Sequential()
    
    # 1st block
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
    # 2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
    # 3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))
    # dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    return model

In [8]:
# Then we need to have a part to train the network:
(x_train, y_train, x_test, y_test) = load_data()
model = build_model()
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])
# train
batch_size = 64
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=EPOCHS, validation_data=(x_test,y_test)) 
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)       

## Keras image augmentation

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)
datagen.fit(x_train)

In [12]:
batch_size = 64
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=EPOCHS,
                    verbose=1,validation_data=(x_test,y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
782/782 [==============================] - 27s 33ms/step - loss: 0.9665 - accuracy: 0.6903 - val_loss: 0.5252 - val_accuracy: 0.8248
Epoch 2/50
782/782 [==============================] - 26s 33ms/step - loss: 0.8033 - accuracy: 0.7232 - val_loss: 0.5361 - val_accuracy: 0.8206
Epoch 3/50
782/782 [==============================] - 26s 33ms/step - loss: 0.7674 - accuracy: 0.7348 - val_loss: 0.5973 - val_accuracy: 0.8022
Epoch 4/50
782/782 [==============================] - 26s 33ms/step - loss: 0.7341 - accuracy: 0.7445 - val_loss: 0.5630 - val_accuracy: 0.8149
Epoch 5/50
782/782 [==============================] - 26s 33ms/step - loss: 0.7212 - accuracy: 0.7497 - val_loss: 0.5271 - val_accuracy: 0.8230
Epoch 6/50
782/782 [==============================] - 26s 33ms/step - loss: 0.7012 - accuracy: 0.7575 - val_loss: 0.5205 - val_accuracy: 0.8267
Epoch 7/50
782/782 [==============================] - 26s 33ms/step - loss: 0.6945 - accuracy: 0.7601 - val_loss: 0.5014 - val_accuracy: